# Unsupervised Lab Session

## Learning outcomes:
- Exploratory data analysis and data preparation for model building.
- PCA for dimensionality reduction.
- K-means and Agglomerative Clustering

## Problem Statement
Based on the given marketing campigan dataset, segment the similar customers into suitable clusters. Analyze the clusters and provide your insights to help the organization promote their business.

## Context:
- Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.
- Customer personality analysis helps a business to modify its product based on its target customers from different types of customer segments. For example, instead of spending money to market a new product to every customer in the company’s database, a company can analyze which customer segment is most likely to buy the product and then market the product only on that particular segment.

## About dataset
- Source: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis?datasetId=1546318&sortBy=voteCount

### Attribute Information:
- ID: Customer's unique identifier
- Year_Birth: Customer's birth year
- Education: Customer's education level
- Marital_Status: Customer's marital status
- Income: Customer's yearly household income
- Kidhome: Number of children in customer's household
- Teenhome: Number of teenagers in customer's household
- Dt_Customer: Date of customer's enrollment with the company
- Recency: Number of days since customer's last purchase
- Complain: 1 if the customer complained in the last 2 years, 0 otherwise
- MntWines: Amount spent on wine in last 2 years
- MntFruits: Amount spent on fruits in last 2 years
- MntMeatProducts: Amount spent on meat in last 2 years
- MntFishProducts: Amount spent on fish in last 2 years
- MntSweetProducts: Amount spent on sweets in last 2 years
- MntGoldProds: Amount spent on gold in last 2 years
- NumDealsPurchases: Number of purchases made with a discount
- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
- NumWebPurchases: Number of purchases made through the company’s website
- NumCatalogPurchases: Number of purchases made using a catalogue
- NumStorePurchases: Number of purchases made directly in stores
- NumWebVisitsMonth: Number of visits to company’s website in the last month

### 1. Import required libraries

In [ ]:
### Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from scipy.stats import zscore
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 2. Load the CSV file (i.e marketing.csv) and display the first 5 rows of the dataframe. Check the shape and info of the dataset.

In [ ]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Datasets/marketing.csv')

# Display the first 5 rows
print(df.head(5))

In [ ]:
#get the info on the data
df.info()

In [ ]:
#get the shape of the data
df.shape

### 3. Check the percentage of missing values? If there is presence of missing values, treat them accordingly.

In [ ]:
#checking if there are missing values
df.isnull().sum()

In [ ]:
#percentage of missing values
df.isnull().sum()/len(df)*100

In [ ]:
#filling the missing value
df['Income'] = df['Income'].fillna(df['Income'].mean())
df.isnull().sum()

### 4. Check if there are any duplicate records in the dataset? If any drop them.

In [ ]:
# check for duplicate data
len(df[df.duplicated()])

There are no duplicate values in the data.


### 5. Drop the columns which you think redundant for the analysis

In [ ]:
# drop the redundant columns
df = df.drop(columns=['ID','Dt_Customer'],axis=1)

### 6. Check the unique categories in the column 'Marital_Status'
- i) Group categories 'Married', 'Together' as 'relationship'
- ii) Group categories 'Divorced', 'Widow', 'Alone', 'YOLO', and 'Absurd' as 'Single'.

In [ ]:
#Before replacing, check the unique values in the Marital_Status column
print(df['Marital_Status'].unique())

In [ ]:
#Replace [Together,Married] as Relationship
df['Marital_Status'] = df['Marital_Status'].replace(['Together','Married'],'Relationship')

#Replace [Divorced,Widow,Alone,YOLO,Absurd] as Single
df['Marital_Status'] = df['Marital_Status'].replace(['Divorced','Widow','Alone','YOLO','Absurd'],'Single')
print(df['Marital_Status'].unique())

After grouping, now there is only two unique values, Single and Relationship

### 7. Group the columns 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', and 'MntGoldProds' as 'Total_Expenses'

In [ ]:
# Group the columns 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', and 'MntGoldProds' as 'Total_Expenses'
df['Total_Expenses'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']

### 8. Group the columns 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', and 'NumDealsPurchases' as 'Num_Total_Purchases'

In [ ]:
# Group the columns 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', and 'NumDealsPurchases' as 'Num_Total_Purchases'
df['Num_Total_Purchases'] = df['NumWebPurchases'] + df['NumCatalogPurchases'] + df['NumStorePurchases'] + df['NumDealsPurchases'] + df['NumWebVisitsMonth']

### 9. Group the columns 'Kidhome' and 'Teenhome' as 'Kids'

In [ ]:
#Group the columsn 'Kidhome' and 'Teenhome' as 'Kids'
df['Kids'] = df['Kidhome'] + df['Teenhome']

### 10. Group columns 'AcceptedCmp1 , 2 , 3 , 4, 5' and 'Response' as 'TotalAcceptedCmp'

In [ ]:
# Group columns 'AcceptedCmp1 , 2 , 3 , 4, 5' and 'Response' as 'TotalAcceptedCmp'
df['TotalAcceptedCmp'] = df['AcceptedCmp1'] + df['AcceptedCmp2'] + df['AcceptedCmp3'] + df['AcceptedCmp4'] + df['AcceptedCmp5'] + df['Response']


### 11. Drop those columns which we have used above for obtaining new features

In [ ]:
df=df.drop(columns=['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds','NumWebVisitsMonth','NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases','Kidhome', 'Teenhome','AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response'],axis=1)
df.head()

### 12. Extract 'age' using the column 'Year_Birth' and then drop the column 'Year_birth'

In [ ]:
from datetime import datetime
df['Age'] = datetime.now().year - df['Year_Birth']

#drop the Year_Birth Column
df.drop('Year_Birth',axis=1,inplace=True)
df.head(3)

### 13. Encode the categorical variables in the dataset

In [ ]:
#Encoding
categorical_data = ['Education','Marital_Status']
lbl = LabelEncoder()
for i in categorical_data:
    df[i] = lbl.fit_transform(df[i])
df.head()

### 14. Standardize the columns, so that values are in a particular range

In [ ]:
df1 = df.copy()
scaled_features = StandardScaler().fit_transform(df1.values)
scaled_features_df = pd.DataFrame(scaled_features, index=df1.index, columns=df1.columns)
scaled_features_df.head(3)

### 15. Apply PCA on the above dataset and determine the number of PCA components to be used so that 90-95% of the variance in data is explained by the same.

In [ ]:
#Generate the Covariance matrix
cov_matrix = np.cov(scaled_features.T)
cov_matrix

In [ ]:
#Calcuate the eigen values and eigen vectors
eig_values, eig_vectors = np.linalg.eig(cov_matrix)
print('eig_values:','/n',eig_values)
print('eig_vectors:','/n',eig_vectors)

In [ ]:
#Calculate explained variance and cumulative variance
total = sum(eig_values)
var_exp = [(i/total)*100 for i in sorted(eig_values,reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print('Explained Variance: ',var_exp)
print('Cummulative Variance Explained: ',cum_var_exp)

In [ ]:
# scree plot
plt.bar(range(10),var_exp,align='center',color='lightgreen',edgecolor='black',label='Explained Variance')
plt.step(range(10),cum_var_exp,where='mid',color='red',label='Cummulative Variance Explained')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.legend(loc='best')
plt.show()

The first 8 principal components are explaining about 95% of the variation.
So the optimal prinicipal componenet is 8.


### 16. Apply K-means clustering and segment the data (Use PCA transformed data for clustering)

In [ ]:
#Using the dimensions obtained from PCA to apply clustering
pca = PCA(n_components=8)
pca_df = pd.DataFrame(pca.fit_transform(scaled_features_df),columns=['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8'])
pca_df.head()

In [ ]:
#Kmeans Clustering using PCA transformed data
#finding optimal K value using elbow plot
cluster_errors = []
cluster_range = range(2,15)
for num_clusters in cluster_range:
    clusters = KMeans(num_clusters,random_state=100)
    clusters.fit(pca_df)
    cluster_errors.append(clusters.inertia_)

#Creating a df of number of clusters and cluster_errors
cluster_df = pd.DataFrame({'num_clusters':cluster_range,'cluster_errors':cluster_errors})

#Elbow plot
plt.figure(figsize=[15,5])
plt.plot(cluster_df['num_clusters'],cluster_df['cluster_errors'],marker='o',color='b')
plt.show()



*   At the inertia drastically decreases.
*   Hence optimal clusters at K=3.


In [ ]:
#Applying KMeans clustering for the optimal number of clusters obtained.
kmeans = KMeans(n_clusters=3, random_state=100)
kmeans.fit(pca_df)

In [ ]:
#creating the df of labels
label = pd.DataFrame(kmeans.labels_,columns=['Label'])

In [ ]:
## joining the label dataframe to the pca_df dataframe
kmeans_df = pca_df.join(label)
kmeans_df.head()

In [ ]:
kmeans_df['Label'].value_counts()

In [ ]:
#visualizing the clusters formed using scatter plot
sns.scatterplot(x=kmeans_df['PC1'],y=kmeans_df['PC2'],hue='Label',data=kmeans_df)
plt.show()

### 17. Apply Agglomerative clustering and segment the data (Use Original data for clustering), and perform cluster analysis by doing bivariate analysis between the cluster label and different features and write your observations.

In [ ]:
plt.figure(figsize=[18,5])
merg = linkage(scaled_features, method='ward')
dendrogram(merg, leaf_rotation=90,)
plt.xlabel('Datapoints')
plt.ylabel('Euclidean distance')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
for i in range(2,15):
    hier = AgglomerativeClustering(n_clusters=i)
    hier = hier.fit(scaled_features_df)
    labels= hier.fit_predict(scaled_features_df)
    print(i,silhouette_score(scaled_features_df,label))



*    The highest score is for clusters 3
*   K=3 is the optimal cluster size




In [ ]:
## Building hierarchical clustering model using the optimal clusters as 3 using original data
hie_cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage = 'ward')
hie_cluster_model = hie_cluster.fit(scaled_features_df)

In [ ]:
## creating a dataframe of the labels
df_label1 = pd.DataFrame(hie_cluster_model.labels_,columns=['Labels'])
df_label1.head(5)

In [ ]:
## joining the label dataframe with unscaled initial cleaned dataframe.(dfc)

df_hier = dfc.join(df_label1)
df_hier.head(5)

### Visualization and Interpretation of results

In [ ]:
sns.barplot(df_hier['Labels'],df_hier['Total_Expenses'])
plt.show()


*   Total Expenses his much higher for cluster 0 as compared to cluster 1 and 2


In [ ]:
sns.barplot(df_hier['Labels'],df_hier['Income'])
plt.show()



*   Cluster 0 is having high income group followed by cluster 2 and cluster 1 is having lower income group






In [ ]:
sns.countplot(df_hier['Marital_Status'],hue='Labels',data=df_hier)
plt.show()



*   We can observe that most of the customers who are in a relationship comes under cluster 0




In [ ]:
sns.barplot(df_hier['Labels'],df_hier['Num_Total_Purchases'])
plt.show()



*   The total number of purchases is also much higher for customers in cluster 0 compared to cluster 1 and 2.


Conclusion:
Customers belonging to cluster 0 does maximum purchases, has spent maximum amount and have maximum income
Where as customers belonging to cluster 1 has the least total expenses , minimum balances and does minimum purchases compared to other 2 clusters
Customers belonging to cluster 2 does average purchanges and has average income




-----
## Happy Learning
-----